# Offline FlexDTW 3D

This notebook provides wrapper functions for calling the [ISA (Iterative Subtractive Alignment) algorithm](https://archives.ismir.net/ismir2021/paper/000101.pdf).  Running this algorithm requires installing some other software, which is described below.  This notebook implements the `offline_processing()` and `online_processing()` functions, which will be imported and run in `02_RunExperiment.ipynb`.

## Offline Processing

The offline processing is the same as in the simple offline DTW system.  In the offline processing stage, three things are computed and stored in the `cache/` folder:
- chroma features for the orchestra recording
- chroma features for the full mix recording
- predicted DTW alignment between the orchestra and full mix recordings



In [1]:
import numpy as np
import pandas as pd
import import_ipynb
import system_utils
import align_tools
import sonify_tools
import os
import os.path
import subprocess
import librosa as lb
from shutil import which
from hmc_mir.align import isa
import matplotlib.pyplot as plt
from numba import jit, njit, prange
import k3d

importing Jupyter notebook from align_tools.ipynb


In [2]:
def offline_processing(scenario_dir, cache_dir, hop_length=2048, downsample=10):
    '''
    Carries out the same offline processing steps as the simple offline DTW system.
    
    Args:
        scenario_dir: The scenario directory to process
        cache_dir: The location of the cache directory
        hop_length: The hop length in samples used when computing chroma features
        downsample: The downsampling factor used when computing chroma_sens features
    
    This function will store the computed chroma features and estimated alignment in the cache folder.
    '''
    # setup
    system_utils.verify_scenario_dir(scenario_dir)
    if os.path.exists(cache_dir):
        # print(f'{cache_dir} has already been processed.  Skipping.')
        pass
    else:
        # setup
        os.makedirs(cache_dir)
        scenario_info = system_utils.get_scenario_info(scenario_dir+'/scenario.info')

        o_file = f'{scenario_dir}/o.wav'
        y_o, sr = lb.core.load(o_file)
        
        orch_start_frm = int(np.round(scenario_info['oStart'] * sr / hop_length))
        orch_end_frm = int(np.round(scenario_info['oEnd'] * sr / hop_length)) + 1

        F_o = lb.feature.chroma_cens(y=y_o, sr=sr, hop_length=hop_length)
        F_o = F_o[:,orch_start_frm:orch_end_frm]
        F_o = F_o[:,::downsample]

        po_file = f'{scenario_dir}/po.wav'
        y_po, sr = lb.core.load(po_file)
        F_po = lb.feature.chroma_cens(y=y_po, sr=sr, hop_length=hop_length)
        F_po = F_po[:,orch_start_frm:orch_end_frm]
        F_po = F_po[:,::downsample]

        np.save(f'{cache_dir}/po_cqt.npy', F_po)
        np.save(f'{cache_dir}/o_cqt.npy', F_o)

    return

In [3]:
def verify_cache_dir(indir):
    '''
    Verifies that the specified cache directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/po_cqt.npy'), f'missing {indir}/po_cqt.npy'
    assert os.path.exists(f'{indir}/o_cqt.npy'), f'missing {indir}/o_cqt.npy'

## Online Processing

### Wrapper Implementation

In [4]:
def online_processing(
    scenario_dir,
    out_dir,
    cache_dir,
    hop_length=2048,
    downsample=10,
    steps=np.array([
        [1,1,1],
        [2,1,1],
        [1,2,1],
        [1,1,2],
        [2,2,1],
        [2,1,2],
        [1,2,2],
    ]),
    weights=np.array([3,4,4,4,5,5,5]),
    buffer = 50
):
    '''
    Carries out `online' processing using the ISA algorithm.
    
    Args:
        scenario_dir: The scenario directory to process
        out_dir: The directory to put results, intermediate files, and logging info
        cache_dir: The cache directory
        hop_length: The hop length in samples used when computing cqt features
        downsample: The downsampling factor used when computing chroma_sens features

    This function will compute and save the predicted alignment in the output directory in a file hyp.npy
    '''
    
    # verify & setup
    system_utils.verify_scenario_dir(scenario_dir)
    verify_cache_dir(cache_dir)
    assert not os.path.exists(out_dir), f'Output directory {out_dir} already exists.'
    os.makedirs(out_dir)
    
    scenario_info = system_utils.get_scenario_info(scenario_dir+'/scenario.info')
    
    p_file = f'{scenario_dir}/p.wav'
    y, sr = lb.core.load(p_file)

    F_p = lb.feature.chroma_cens(y=y, sr=sr, hop_length=hop_length)[:,::downsample]
    F_po = np.load(f'{cache_dir}/po_cqt.npy') # full mix features
    F_o = np.load(f'{cache_dir}/o_cqt.npy') # orchestra

    hop_sec = downsample * hop_length / sr

    # compute the alignment
    best_cost, path = flexdtw(F_p, F_o, F_po, steps, weights, buffer=buffer)

    wp_AC = np.vstack((path[0],path[1]))
    wp_AC_sec = wp_AC*hop_sec
    wp_AC_sec[1,:] += scenario_info['oStart']
    np.save(f'{out_dir}/hyp.npy', wp_AC_sec)
    
    return

In [5]:
def verify_hyp_dir(indir):
    '''
    Verifies that the specified scenario hypothesis directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/hyp.npy'), f'{indir} is missing the required files, please re run the online processing'

# Example

Here is an example of how to call the offline and online processing functions on a scenario directory.

In [6]:
# scenario_dir = 'scenarios/s105'
# out_dir = 'experiments/flexDTW/s105'
# cache_dir = 'experiments/flexDTW/cache/beeth1_mov1_O1_PO1'
# hop_length = 2048
# downsample = 10
# scenario_info = system_utils.get_scenario_info(scenario_dir+'/scenario.info')

In [7]:
# orch_start_frm = int(np.round(scenario_info['oStart'] * sr / hop_length))
# orch_end_frm = int(np.round(scenario_info['oEnd'] * sr / hop_length)) + 1

# o_file = f'{scenario_dir}/o.wav'
# y_o, sr = lb.core.load(o_file)
# # y_o = y_o[int(orch_start_sec*sr):int(orch_end_sec*sr)]
# F_o = lb.feature.chroma_cens(y=y_o, sr=sr, hop_length=hop_length)
# F_o = F_o[:,orch_start_frm:orch_end_frm]
# F_o = F_o[:,::downsample]

# po_file = f'{scenario_dir}/po.wav'
# y_po, sr = lb.core.load(po_file)
# # y_po = y_po[int(orch_start_sec*sr):]
# F_po = lb.feature.chroma_cens(y=y_po, sr=sr, hop_length=hop_length)
# F_po = F_po[:,orch_start_frm:orch_end_frm]
# F_po = F_po[:,::downsample]

# np.save(f'{cache_dir}/po_cqt.npy', F_po)
# np.save(f'{cache_dir}/o_cqt.npy', F_o)

In [8]:
# p_file = f'{scenario_dir}/p.wav'
# y, sr = lb.core.load(p_file)

# F_p = lb.feature.chroma_cens(y=y, sr=sr, hop_length=hop_length)[:,::downsample]
# F_po = np.load(f'{cache_dir}/po_cqt.npy') # full mix features
# F_o = np.load(f'{cache_dir}/o_cqt.npy') # orchestra

# hop_sec = downsample * hop_length / sr

In [9]:
# F_p.shape, F_o.shape, F_po.shape

In [10]:
@njit(parallel=True)
def calculate_cost_tensor(x, y, z):
    '''
    Calculates the cost tensor for the given piano, orchestra, and piano-orchestra features.
    
    Args:
        x: submix features
        y: submix features
        z: full mix features
    '''
    x = x.T
    y = y.T
    z = z.T
    cost_tensor = np.zeros((x.shape[0], y.shape[0], z.shape[0]))
    for i in prange(x.shape[0]):
        for j in prange(y.shape[0]):
            for k in prange(z.shape[0]):
                cost_tensor[i, j, k] = 1 - (np.dot(z[k,:], x[i,:] + y[j,:]) / (np.linalg.norm(z[k,:]) * np.linalg.norm(x[i,:] + y[j,:])))
    return cost_tensor

In [11]:
@jit(nopython=True)
def backtrace_flexdtw(D, B, steps, rstart, cstart, tstart):
    '''
    Backtraces through the cumulative cost matrix D starting from a specified location.
    
    Arguments:
    D: cumulative cost matrix
    B: backtrace matrix
    steps: a numpy matrix specifying the allowable transitions.  It should be of
            dimension (L, 2), where each row specifies (row step, col step)
    rstart: the row index to start backtracking from
    cstart: the column index to start backtracking from
    
    Outputs
    path: a python list of (row, col) coordinates for the optimal path.
    '''
    pos = (rstart, cstart, tstart)
    path = []
    path.append(pos)
    while(pos[0] != 0 and pos[1] != 0 and pos[2] != 0):
        (row, col, tube) = pos
        stepidx = B[row, col, tube]
        (rstep, cstep, tstep) = steps[stepidx]
        pos = (row-rstep, col-cstep, tube-tstep)
        path.append(pos)
    
    return path

In [12]:
@jit(nopython=True)
def find_best_endpoint(D, P, buffer):
    '''
    Determines the best location to begin backtracking from by comparing the average path cost
    per manhattan block.
    
    Inputs
    D: the cumulative cost matrix
    P: the matrix specifying the starting location of the alignment path
    buffer: specifies the length of a buffer region (in frames) to avoid short degenerate alignment paths
        near the corners of the pairwise cost matrix.  This can be thought of as the minimum length that
        needs to match in order to be considered a valid alignment path.
    
    Outputs
    best_cost: the best average path cost per manhattan block
    best_r: the row index of the best endpoint
    best_c: the column index of the best endpoint
    debug: debugging information for examining the average cost per manhattan block for each 
        of the candidate ending positions
    '''
    
    # consider the top corner 3 faces as candidates
    candidates = []
    for i in range(buffer, D.shape[0]):
        for j in range(buffer, D.shape[1]):
            candidates.append((i, j, D.shape[2]-1))

    for j in range(buffer, D.shape[1]):
        for k in range(buffer, D.shape[2]):
            candidates.append((D.shape[0]-1, j, k))

    for i in range(buffer, D.shape[0]):
        for k in range(buffer, D.shape[2]):
            candidates.append((i, D.shape[1]-1, k))
    
    best_cost = np.inf
    best_r, best_c, best_t = -1, -1, -1
    # debug = []
    debug = np.zeros_like(D)
    
    for i, (r,c,t) in enumerate(candidates):
                
        # get alignment start location
        rstart, cstart, tstart = P[r,c,t]
            
        # calculate average cost per manhattan block
        mdist = (r - rstart) + (c - cstart) + (t - tstart)# manhattan distance
        avg_cost_per_mb = D[r,c,t] / mdist
        
        # keep best
        if avg_cost_per_mb < best_cost:
            best_cost = avg_cost_per_mb
            best_r, best_c, best_t = r, c, t
            
        # debugging info
        # TODO: updated this for 3D-FlexDTW
        # if r == D.shape[0]-1:
        #     debug.append((c-D.shape[1]+1, avg_cost_per_mb, r, c))
        # else:
        #     debug.append((D.shape[0]-1-r, avg_cost_per_mb, r, c))
        debug[r,c,t] = avg_cost_per_mb
    
    return best_cost, best_r, best_c, best_t, debug

In [13]:
@jit(nopython=True)
def flexdtw(x, y, z, steps, weights, buffer = 1):
    C = calculate_cost_tensor(x, y, z)
    
    # Run FlexDTW on the cost tensor
    D = np.ones(C.shape) * np.inf
    B = np.zeros(C.shape, dtype=np.int8)

    # D[0, :, :] = np.inf
    # D[:, 0, :] = np.inf
    # D[:, :, 0] = np.inf
    # for row in range(1,C.shape[0]):
    #     for col in range(1, C.shape[1]):
    #         for tube in range (1, C.shape[2]):
    #             D[row, col, tube] = -np.inf
    D[0,0,0] = C[0,0,0]

    # DP
    for row in range(1,C.shape[0]):
        for col in range(1, C.shape[1]):
            for tube in range (1, C.shape[2]):
                mincost = np.inf
                minidx = -1
                bestrprev = -1
                bestcprev = -1
                besttprev = -1
                for stepidx, step in enumerate(steps):
                    (rstep, cstep, tstep) = step
                    prevrow = row - rstep
                    prevcol = col - cstep
                    prevtube = tube - tstep
                    if prevrow >= 0 and prevcol >= 0 and prevtube >= 0:
                        
                        # calculate avg cost per manhattan block
                        pathcost = D[prevrow, prevcol, prevtube] + C[row, col, tube] * weights[stepidx]
                        
                        # select best transition based on avg cost per manhattan block
                        if pathcost < mincost:
                            mincost = pathcost
                            minidx = stepidx
                            bestrprev = prevrow
                            bestcprev = prevcol
                            besttprev = prevtube
                            
                D[row, col, tube] = D[bestrprev, bestcprev, besttprev] + C[row, col, tube] * weights[minidx]
                B[row, col, tube] = minidx

    best_cost = D[-1, -1, -1]
    path = backtrace_flexdtw(D, B, steps, x.shape[1]-1, y.shape[1]-1, z.shape[1]-1)
    path.reverse()
    path = np.array(path)
    return best_cost, path.T


In [14]:
# best_cost, path = flexdtw(F_p, F_o, F_po, np.array([
#         [1,1,1],
#         [2,1,1],
#         [1,2,1],
#         [1,1,2],
#         [2,2,1],
#         [2,1,2],
#         [1,2,2],
#     ]),
#     np.array([3,4,4,4,5,5,5]), buffer = 50)

In [15]:
# plt_line = k3d.line(path.T, width=0.1, color=0xff99cc)

# start_label = k3d.label('0,0,0', position=(0, 0, 0))
# # end_label = k3d.label('{}, {}, {}'.format(x.shape[1], y.shape[1], z.shape[1]), position=(x.shape[1], y.shape[1], z.shape[1]))
# end_label = k3d.label('{}, {}, {}'.format(F_p.shape[1], F_o.shape[1], F_po.shape[1]), position=(F_p.shape[1], F_o.shape[1], F_po.shape[1]))
# # end_label = k3d.label('{}, {}, {}'.format(context_size, context_size, context_size), position=(context_size, context_size, context_size))

# plot = k3d.plot()
# plot += plt_line
# plot += start_label
# plot += end_label
# plot.display()

In [16]:
# cost_tensor = calculate_cost_tensor(F_p, F_o, F_po)

In [17]:
# plt.imshow(cost_tensor[:,0,:], origin='lower')
# plt.colorbar(orientation='horizontal')
# plt.show()

In [18]:
# cost_tensor.shape

In [19]:
# plt_volume = k3d.volume(cost_tensor, bounds=[0, cost_tensor.shape[0], 0, cost_tensor.shape[1], 0, cost_tensor.shape[2]])

# plot = k3d.plot()
# plot += plt_volume
# plot.display()

In [20]:
# vol = debug
# plt_volume = k3d.volume(vol, bounds=[0, vol.shape[0], 0, vol.shape[1], 0, vol.shape[2]])
# # plt_volume = k3d.volume(calculate_cost_tensor(F_p, F_o, F_po))

# plot = k3d.plot()
# plot += plt_volume
# plot.display()

In [21]:
# # infer piano-orchestra alignment
# wp_AC = np.vstack((path[0],path[1]))
# wp_AC_sec = wp_AC*hop_sec
# wp_AC_sec[1,:] += scenario_info['oStart']
# np.save(f'{out_dir}/hyp.npy', wp_AC_sec)

In [22]:
# np.vstack((path[0],path[1]))

In [23]:
# system_utils.get_orchestra_query_boundaries(scenario_dir)

In [24]:
# # import matplotlib.pyplot as plt

# # alignment1 = wp_AB_
# # alignment2 = wp_BC
# # alignment3 = wp_AC

# # plt.plot(path[0], path[2], label='AB')
# # plt.plot(path[1], path[2], label='BC')
# plt.plot(path[1], path[0], label='AC')
# plt.xlabel('Source Time')
# plt.ylabel('Aligned Time')
# plt.ylim(ymin=0)
# plt.xlim(xmin=0)
# plt.legend()
# plt.show()